In [33]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read Local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

In [34]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [35]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [36]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are MamakBot, an automated service at a mamak restaurant.You speak like mamak in informal malay \
You warmly greet customers, take their orders, provide options, \
and ask if they want any add-ons. \
After confirming the order, you inquire if they will be dining in \
or taking away. You wait for the complete order, summarize it, and \
double-check if they need anything else. \
If they are dining in, you ask for a table preference. \
Finally, you collect the payment. \
Make sure to explain menu items, choices, and sizes clearly to avoid any confusion. \
The menu includes: \
nasi lemak 6.95 \
roti canai 1.50 \
teh tarik 2.00 \
maggi goreng 5.00 \
roti telur 2.50 \
roti bakar 1.75 \
Toppings: \
telur mata 0.50 \
sambal 0.75 \
daging 2.50 \
ayam 2.00 \
Drinks: \
teh o ais 2.50 \
limau ais 2.00 \
air bandung 2.75 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, height=300, loading_indicator=True)

In [37]:
import panel as pn
pn.extension()

# Define the context
context = [
    {
        'role': 'system',
        'content': """
        You are MamakBot, an automated service at a mamak restaurant.
        You speak informally like a mamak in Malay.
        You warmly greet customers, take their orders, provide options,
        and ask if they want any add-ons.
        After confirming the order, you inquire if they will be dining in
        or taking away. You wait for the complete order, summarize it, and
        double-check if they need anything else.
        If they are dining in, you ask for a table preference.
        Finally, you collect the payment.
        Make sure to explain menu items, choices, and sizes clearly to avoid any confusion.
        The menu includes:
        nasi lemak 6.95
        roti canai 1.50
        teh tarik 2.00
        maggi goreng 5.00
        roti telur 2.50
        roti bakar 1.75
        Toppings:
        telur mata 0.50
        sambal 0.75
        daging 2.50
        ayam 2.00
        Drinks:
        teh o ais 2.80
        limau ais 2.00
        air bandung 2.75
        milo ais 3.00
        """
    }
]

# Create widgets
input_text = pn.widgets.TextInput(
    value="Hi",
    placeholder='Enter text here…',
    sizing_mode='stretch_width'
)

button_conversation = pn.widgets.Button(name="Chat!", button_type="primary")

# Bind the conversation function to the button
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard layout
dashboard = pn.Column(
    pn.pane.Markdown("## MamakBot - Automated Mamak Restaurant Service", style={'font-size': '20px'}),
    pn.Row(input_text, button_conversation, align='center', margin=(10, 0)),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
    pn.Row(button_convert_to_json, align='end', margin=(10, 0)),  # Add the JSON conversion button
    sizing_mode='stretch_both'

)

# Define custom CSS styles
custom_css = """
.beautiful-dashboard {
    background-color: #f5f5f5;
    padding: 20px;
}
"""

# Set custom CSS using pn.config.raw_css
pn.config.raw_css.append(custom_css)

# Display the dashboard
dashboard.servable()


Column(sizing_mode='stretch_both')
    [0] Markdown(str, style={'font-size': '20px'})
    [1] Row(align='center', margin=(10, 0))
        [0] TextInput(placeholder='Enter text here…', sizing_mode='stretch_width', value='Hi')
        [1] Button(button_type='primary', name='Chat!')
    [2] ParamFunction(function, height=300, loading_indicator=True)
    [3] Row(align='end', margin=(10, 0))
        [0] Button(button_type='success', name='Convert to JSON')

In [38]:
def convert_to_json():
    messages =  context.copy()
    messages.append(
    {'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
     The fields should be 1) food 2) list of toppings 3) list of drinks  4) list of sides include size  5)total price '},    
    )
     #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

    response = get_completion_from_messages(messages, temperature=0)

    return response

In [39]:
print(convert_to_json)

<function convert_to_json at 0x000002B21E476F70>


In [40]:
button_convert_to_json = pn.widgets.Button(name="Convert to JSON", button_type="success")

In [41]:
button_convert_to_json.on_click(convert_to_json)

Watcher(inst=Button(button_type='success', name='Convert to JSON'), cls=<class 'panel.widgets.button.Button'>, fn=<function convert_to_json at 0x000002B21E476F70>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [ ]:
xxxxxxxxxxxxxx